#1. Funciones auxiliares y lectura de datos


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
print("hola Elias")
def leer_rango_excel_a_array(ruta_archivo, hoja_nombre=0, columnas_a_leer=""):
    """
    Abre un archivo Excel y extrae un rango de celdas específico
    (D3 a BO42) y lo guarda en un array (lista de listas).
"""
    print(f"Intentando leer el archivo: {ruta_archivo}...")

    try:
        # 1. Filas:
        # Queremos empezar en la Fila 3.
        # Las filas en pandas se indexan desde 0.
        # Fila 1 = índice 0, Fila 2 = índice 1, Fila 3 = índice 2.
        # Así que saltamos las 2 primeras filas.
        filas_a_saltar = 2  # (Saltamos Fila 1 y Fila 2)
        filas_a_leer = 40

        # Queremos leer desde la fila 3 hasta la 42 (inclusive).
        # Número de filas = 42 - 3 + 1 = 40 filas.

        # --- Lectura del archivo ---

        # Usamos 'header=None' porque el rango (D3) no es un encabezado,
        # es el inicio de los datos.
        df_rango = pd.read_excel(
            ruta_archivo,
            sheet_name= hoja_nombre, #Segunda hoja, Espectro Total
            header=None,             # Trata la primera fila leída (Fila 3) como datos
            skiprows=filas_a_saltar, # Salta las 2 primeras filas
            usecols=columnas_a_leer, # Lee solo el rango de columnas D-BO
            nrows=filas_a_leer       # Lee exactamente 40 filas
        )

        # Convertir el DataFrame de pandas a un array (lista de listas)
        # .values obtiene los datos como un array de NumPy
        # .tolist() lo convierte a una lista estándar de Python
        array_valores = df_rango.values.tolist()

        # Verificación
        if array_valores:
            num_filas = len(array_valores)
            num_cols = len(array_valores[0]) # Longitud de la primera fila
            print(f"Éxito: Se leyó un array de {num_filas} filas x {num_cols} columnas.")
        else:
            print("Advertencia: El rango D3:BO42 parece estar vacío.")

        return array_valores

    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo en la ruta: '{ruta_archivo}'.")
        return None
    except Exception as e:
        print(f"Ocurrió un error inesperado al leer el rango: {e}")
        return None

def separar_filas_en_matrices(datos_array):
    """
    Toma una lista de listas (matriz) y la separa en dos matrices:
    - Una con las filas de índice par (0, 2, 4...)
    - Otra con las filas de índice impar (1, 3, 5...)
    """

    # Listas para guardar las *filas* (que son listas)
    matriz_filas_impares_excel = []
    matriz_filas_pares_excel = []

    for i, fila in enumerate(datos_array):

        # i % 2 == 0 significa índice 0, 2, 4... (Filas Excel 3, 5, 7...)
        if i % 2 == 0:
            # --- CAMBIO CLAVE ---
            # Añade la 'fila' (que es una lista) como un nuevo elemento
            matriz_filas_impares_excel.append(fila)

        # else significa índice 1, 3, 5... (Filas Excel 4, 6, 8...)
        else:
            # --- CAMBIO CLAVE ---
            matriz_filas_pares_excel.append(fila)

    return matriz_filas_impares_excel, matriz_filas_pares_excel



hola Elias
